## 딥러닝 모델의 윤리 및 책임감 있는 AI 개발

### 내용
1. AI와 딥러닝 모델의 윤리적 문제
    - AI 윤리
        - AI와 딥러닝 모델의 사회적 영향력
        - 윤리적 문제와 책임감 있는 AI 개발의 필요성

    - 주요 윤리적 문제
        - 편향성(Bias)
        - 프라이버시(Privacy)
        - 투명성(Transparency)
        - 공정성(Fairness)
        - 신뢰성(Reliability)

- 예제 나중에

## 최신 딥러닝 연구 및 기술 동향

### 강의내용
1. 최신 딥러닝 연구 동향

2. 자연어 처리(NLP) 최신 기술

    - Transformer 기반 모델
        - Transformer 개념 및 구조
        - BERT, GPT 등의 최신 언어 모델

In [ ]:
# BERT를 활용한 문서 분류 예제

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

# BERT 모델과 토크나이저 로드
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# 데이터셋 준비
texts = ["I love this movie!", "This was a terrible film."]
labels = [1, 0]
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=inputs,
    eval_dataset=inputs
)

# 모델 훈련
trainer.train()

3. 생성 모델(Generative Models) 최신 기술

    - GAN 및 변이형 오토인코더(VAE)
        - GAN 개념 및 구조
        - VAE 개념 및 구조

    - VAE를 활용한 이미지 생성 예제

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# VAE 모델 정의
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = torch.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h3 = torch.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# 손실 함수 정의
def loss_function(recon_x, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# 데이터셋 로드 및 전처리
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

# 모델, 옵티마이저 설정
model = VAE()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 모델 학습
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for data, _ in train_loader:
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {train_loss/len(train_loader.dataset)}')

4. 강화 학습(Reinforcement Learning) 최신 기술

    - 강화 학습 개념 및 주요 알고리즘
        - DQN, PPO, A3C 등의 최신 강화 학습 알고리즘
    - Proximal Policy Optimization(PPO) 알고리즘 예제

In [ ]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

# 환경 설정
env = gym.make('CartPole-v1')

# PPO 모델 정의
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine = nn.Linear(4, 128)
        self.action_head = nn.Linear(128, 2)
        self.value_head = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.affine(x))
        action_scores = self.action_head(x)
        state_values = self.value_head(x)
        return torch.softmax(action_scores, dim=-1), state_values

# PPO 에이전트 정의
class PPOAgent:
    def __init__(self):
        self.model = Policy()
        self.optimizer = optim.Adam(self.model.parameters(), lr=1e-2)
        self.eps_clip = 0.2
        self.gamma = 0.99

    def select_action(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0)
        probs, _ = self.model(state)
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

    def update(self, rewards, log_probs, state_values):
        returns = []
        R = 0
        for r in rewards[::-1]:
            R = r + self.gamma * R
            returns.insert(0, R)
        returns = torch.tensor(returns)
        log_probs = torch.cat(log_probs)
        state_values = torch.cat(state_values)
        advantages = returns - state_values

        policy_loss = []
        value_loss = []
        for log_prob, value, advantage in zip(log_probs, state_values, advantages):
            ratio = torch.exp(log_prob - log_prob.detach())
            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1 - self.eps_clip, 1 + self.eps_clip) * advantage
            policy_loss.append(-torch.min(surr1, surr2))
            value_loss.append((returns - value).pow(2))

        self.optimizer.zero_grad()
        loss = torch.stack(policy_loss).sum() + torch.stack(value_loss).sum()
        loss.backward()
        self.optimizer.step()

# PPO 학습
agent = PPOAgent()
num_episodes = 1000
for i_episode in range(num_episodes):
    state = env.reset()
    log_probs = []
    state_values = []
    rewards = []
    for t in range(1000):
        action, log_prob = agent.select_action(state)
        state, reward, done, _ = env.step(action)
        rewards.append(reward)
        log_probs.append(log_prob)
        state_values.append(agent.model(torch.from_numpy(state).float().unsqueeze(0))[1])
        if done:
            break
    agent.update(rewards, log_probs, state_values)
    if i_episode % 100 == 0:
        print(f'Episode {i_episode}, Total Reward: {sum(rewards)}')